In [15]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from itertools import combinations_with_replacement,product
from sklearn.metrics import accuracy_score

from keras.models import Model
from keras.layers import Dense,Input
from keras.layers.convolutional import Conv2D, AtrousConvolution2D
import keras
import keras.backend as K

In [2]:
epochs = 100
range_min = 0
range_max = 9
batch_size = 4 
num_classes = 82
num_train_samples = 100

In [3]:
x = np.array(list(product(range(10),repeat = 2)))
np.random.shuffle(x)
y = x[:,0] * x[:,1]
train_labels = torch.from_numpy(y).type(torch.LongTensor)
train_data = torch.from_numpy(x).type(torch.FloatTensor)

In [33]:
def net(inputs):
    input_data = Input(shape = inputs)
    out = Dense(30,activation = 'relu',name='fc1')(input_data)
    out = Dense(82,activation='softmax',name='fc2')(out)
    
    return Model(inputs = input_data,outputs = out)

In [34]:
model_mul = net((2,))
model_mul.trainable = False
model_mul.compile(loss = 'categorical_crossentropy',optimizer = 'Adam',metrics = ['accuracy'])
# model_mul.summary()
model_mul.load_weights('../mul_0-9_classifier.h5')

In [43]:
def keras_to_pyt(km, pm):
    weight_dict = dict()
    for layer in km.layers:
        if type(layer) is keras.layers.convolutional.Conv2D:
            weight_dict[layer.get_config()['name'] + '.weight'] = np.transpose(layer.get_weights()[0], (3, 2, 0, 1))
            weight_dict[layer.get_config()['name'] + '.bias'] = layer.get_weights()[1]
        elif type(layer) is keras.layers.Dense:
            weight_dict[layer.get_config()['name'] + '.weight'] = np.transpose(layer.get_weights()[0], (1, 0))
            weight_dict[layer.get_config()['name'] + '.bias'] = layer.get_weights()[1]
    pyt_state_dict = pm.state_dict()
    for key in pyt_state_dict.keys():
        pyt_state_dict[key] = torch.from_numpy(weight_dict[key])
    pm.load_state_dict(pyt_state_dict)
    return (pm,pyt_state_dict)

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(2,30)
        self.fc2 = nn.Linear(30,82)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x
    
    def predict(self,x):
        pred = self.forward(x)
        _,out = torch.max(pred,1)
        return out

In [54]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
for epoch in range(epochs):
    running_loss = 0.0 
    batch_min = 0
    batch_max = batch_size
    for batch in range(num_train_samples // batch_size):
        data = train_data[batch_min:batch_min+batch_size]
        labels = train_labels[batch_min:batch_min+batch_size]
        
        optimizer.zero_grad()
        out = net.forward(data)
        loss = criterion(out,labels)
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
    
    batch_min += batch_size
    batch_max += batch_size
    print("epoch : ",epoch, "   loss : ",running_loss)

In [55]:
net,weights = keras_to_pyt(model_mul, net)

In [57]:
torch.save(net,'mul_0_9')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [58]:
model1 = torch.load('mul_0_9')

In [59]:
pred = model1.forward(train_data).exp().detach()
_,indices = torch.max(pred,1)
indices = indices.numpy()
print(indices)
print(train_labels)

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


[ 5 72 25 63 20  0 12  2  0  0  0 28 45 12 18 24 63  4 54 18  4 21  0 27
  0  0 48 18  0  0 24  0  8  6 28  9  6 24 36 12  0 20  7  8 21 18 32  8
 36 54  0 30  0 16 16 15  7  0  6  6 24 56 49  2 45 40 32 14  4 42 48 10
 72  0 30 64 40 56 15 14  9  0 10  1 16  0 81  0  3 36 35  9 35  8  5 27
 42  0 12  3]
tensor([ 5, 72, 25, 63, 20,  0, 12,  2,  0,  0,  0, 28, 45, 12, 18, 24, 63,  4,
        54, 18,  4, 21,  0, 27,  0,  0, 48, 18,  0,  0, 24,  0,  8,  6, 28,  9,
         6, 24, 36, 12,  0, 20,  7,  8, 21, 18, 32,  8, 36, 54,  0, 30,  0, 16,
        16, 15,  7,  0,  6,  6, 24, 56, 49,  2, 45, 40, 32, 14,  4, 42, 48, 10,
        72,  0, 30, 64, 40, 56, 15, 14,  9,  0, 10,  1, 16,  0, 81,  0,  3, 36,
        35,  9, 35,  8,  5, 27, 42,  0, 12,  3])


In [60]:
model1.predict(train_data)

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


tensor([ 5, 72, 25, 63, 20,  0, 12,  2,  0,  0,  0, 28, 45, 12, 18, 24, 63,  4,
        54, 18,  4, 21,  0, 27,  0,  0, 48, 18,  0,  0, 24,  0,  8,  6, 28,  9,
         6, 24, 36, 12,  0, 20,  7,  8, 21, 18, 32,  8, 36, 54,  0, 30,  0, 16,
        16, 15,  7,  0,  6,  6, 24, 56, 49,  2, 45, 40, 32, 14,  4, 42, 48, 10,
        72,  0, 30, 64, 40, 56, 15, 14,  9,  0, 10,  1, 16,  0, 81,  0,  3, 36,
        35,  9, 35,  8,  5, 27, 42,  0, 12,  3])